In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/Thesis

/content/drive/MyDrive/Thesis


In [ ]:
!pip3 install ./fairseq/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing ./fairseq
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for fairseq: filename=fairseq-0.10.2-cp37-cp37m-linux_x86_64.whl size=2775513 sha256=d9a10241730207f4a9d29ad260b0d4dfc70e12d1752dd6939125be1e3bd16c48
  Stored in directory: /tmp/pip-ephem-wheel-cache-64pxybsz/wheels/b8/35/c4/f1ecf9b4b671c068b9b4f286546e7d60fe7cad36ecc5acb3ba
Successfully built fair

In [ ]:
!pip install sacrebleu==1.5.1
!pip install sentencepiece
!pip install tensorboardX

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached sacrebleu-1.5.1-py3-none-any.whl (54 kB)
  Attempting uninstall: sacrebleu
    Found existing installation: sacrebleu 2.1.0
    Uninstalling sacrebleu-2.1.0:
      Successfully uninstalled sacrebleu-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 1.1.2 requires sacrebleu>=2.0.0, but you have sacrebleu 1.5.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%cd contextual-mt/
!pip3 install -e .

/content/drive/MyDrive/Thesis/contextual-mt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/drive/MyDrive/Thesis/contextual-mt
  Attempting uninstall: contextual-mt
    Found existing installation: contextual-mt 0.0.1
    Can't uninstall 'contextual-mt'. No files were found to uninstall.
  Running setup.py develop for contextual-mt


In [ ]:
!bash cxmi.sh

CXMI: 0.00161


In [ ]:
!bash inference.sh

  0% 0/1268 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size
100% 1268/126

In [ ]:
with open("/content/drive/MyDrive/Thesis/contextual-mt/predictions/test.pred.tgt") as hyp_f:
    hyps = [line.strip() for line in hyp_f.readlines()]
with open("/content/drive/MyDrive/Thesis/contextual-mt/projects/tir1/corpora/dialogue_mt/iwslt2015/en-vi/test.en-vi.vi") as ref_f:
    refs = [line.strip() for line in ref_f.readlines()]
with open("/content/drive/MyDrive/Thesis/contextual-mt/projects/tir1/corpora/dialogue_mt/iwslt2015/en-vi/test.en-vi.en") as src_f:
    srcs = [line.strip() for line in src_f.readlines()]

In [ ]:
import sacrebleu
assert len(hyps) == len(refs)
print(sacrebleu.corpus_bleu(hyps, [refs]).format())

BLEU = 34.42 65.3/40.1/28.0/19.1 (BP = 1.000 ratio = 1.024 hyp_len = 40654 ref_len = 39697)


In [ ]:
comet_input = [
            {"src": src, "mt": mt, "ref": ref} for src, mt, ref in zip(srcs, hyps, refs)
        ]

In [ ]:
comet_input[17]

{'mt': 'Tôi luôn tự hỏi tại_sao họ có ánh_sáng nhưng không.',
 'ref': 'Tôi lúc_nào cũng tự hỏi là tại_sao họ lại có điện còn chúng_tôi thì không.',
 'src': "I always wondered why they had lights but we didn't."}

In [ ]:
!pip install unbabel-comet==1.1.2 --use-feature=2020-resolver

In [ ]:
from comet import download_model, load_from_checkpoint

model_path = download_model("wmt20-comet-da")
model = load_from_checkpoint(model_path)


In [ ]:
seg_scores, sys_score = model.predict(comet_input, batch_size=8, gpus=1)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 159/159 [00:22<00:00,  7.09it/s]


In [ ]:
print(f"COMET = {sum(seg_scores)/len(seg_scores):.4f}")

COMET = 0.2681


In [ ]:
print(f"COMET = {sys_score:.4f}")

COMET = 0.2681
